In [2]:
from ladybug.sql import SQLiteResult
from ladybug.analysisperiod import AnalysisPeriod as ap
import numpy as np
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots


import sys
sys.path.insert(0, "../scripts")
from helpers import *

## Testing Converting Rosse Room Model to AFN 

### test 02 -> change geo and have afn 

In [12]:
dir = "../rosseRoomModel/230303_rr/test03_zone_vent/eplusout.sql"
t02 = LoadSQL(dir, "Test 03")

In [13]:
t02.ao
t02.enumerate_data_headers("Site Outdoor Air Drybulb Temperature")

des_data = [(i,j,k) for i, j,k, in zip(["Zone Mean Air Temperature"]*2 + ["Site Outdoor Air Drybulb Temperature"], [0,1,0], ["Zone A", "Zone B", "Out"])]

t02d = t02.get_valid_data(des_data)

t02.plot_temps()

0 
 Temperature (C)
9/21 to 9/21 between 0 and 23 @4
type: Site Outdoor Air Drybulb Temperature
Zone: Environment 

1 
 Temperature (C)
1/21 to 1/21 between 0 and 23 @4
type: Site Outdoor Air Drybulb Temperature
Zone: Environment 

i continued, Zone A
i continued, Zone B
i continued, Out


### test01 -> just change the geo 

In [11]:
dir = "../rosseRoomModel/230301_rr/test_01/eplusout.sql"
t01= LoadSQL(dir, "Test 01")

In [7]:

t02.enumerate_data_headers("Site Outdoor Air Drybulb Temperature")

0 
 Temperature (C)
9/21 to 9/21 between 0 and 23 @4
type: Site Outdoor Air Drybulb Temperature
Zone: Environment 

1 
 Temperature (C)
1/21 to 1/21 between 0 and 23 @4
type: Site Outdoor Air Drybulb Temperature
Zone: Environment 



In [8]:
t02.ao 

('Zone Mean Air Temperature',
 'Site Wind Direction',
 'Site Outdoor Air Drybulb Temperature',
 'Site Wind Speed')

In [13]:
t01d = t01.get_valid_data(des_data)

t01d

{'Zone A': 15 Minute Continuous Data Collection
 9/21 to 9/21 between 0 and 23 @4
 Temperature (C)
 ...96 values...,
 'Zone B': 15 Minute Continuous Data Collection
 9/21 to 9/21 between 0 and 23 @4
 Temperature (C)
 ...96 values...,
 'Out': 15 Minute Continuous Data Collection
 9/21 to 9/21 between 0 and 23 @4
 Temperature (C)
 ...96 values...}

In [14]:
t01.plot_temps()

i continued, Zone A
i continued, Zone B
i continued, Out


### comparing to original to original rosse mode, but with new dates matching afn exp

In [15]:
dir = "../rosseRoomModel/230301_rr/rosse_01/eplusout.sql"
rosse01 = LoadSQL(dir)
rosse01.ao

('Site Outdoor Air Drybulb Temperature',
 'Site Outdoor Air Wetbulb Temperature',
 'Site Diffuse Solar Radiation Rate per Area',
 'Site Outdoor Air Drybulb Temperature',
 'Site Direct Solar Radiation Rate per Area',
 'People Air Temperature',
 'People Convective Heating Energy',
 'Zone Air Temperature',
 'Zone Air Relative Humidity',
 'Electricity:Facility',
 'People Total Heating Energy',
 'People Air Relative Humidity',
 'People Total Heating Rate',
 'People Occupant Count',
 'Electricity:Facility',
 'Surface Inside Face Temperature',
 'Surface Outside Face Temperature')

In [17]:
rosse01.enumerate_data_headers("Site Outdoor Air Drybulb Temperature")
# TODO not sure why this isnt working but return to this later 

ValueError: range() arg 3 must not be zero

## Single Sided Ventilation 

In [3]:
dir0 = "../examples/AFN_SS/eplusout.sql"
a = LoadSQL(dir0, "Single Sided AFN")

In [11]:
a.ao

('Site Wind Speed',
 'AFN Linkage Node 1 to Node 2 Pressure Difference',
 'Site Outdoor Air Drybulb Temperature',
 'AFN Linkage Node 2 to Node 1 Mass Flow Rate',
 'Site Wind Direction',
 'Zone Mean Air Temperature',
 'AFN Linkage Node 1 to Node 2 Mass Flow Rate')

In [5]:
desired_data = [(i, j, k) for i, j, k in zip(["Zone Mean Air Temperature"]*4 + ["Site Outdoor Air Drybulb Temperature", "Site Wind Direction"], [4,5,6,7,1, 0], ["West", "South", "North", "East", "Sep Out", "Wind"] )]
ad = a.get_valid_data(desired_data)
ad.keys()

dict_keys(['West', 'South', 'North', 'East', 'Sep Out', 'Wind'])

In [6]:
a.plot_temps()

i continued, West
i continued, South
i continued, North
i continued, East
i continued, Sep Out
This doesn't have temperature units


## AFN Occupant/Temperature Controlled Window 

In [3]:
dir = "../examples/AFNWindowOp/eplusout.sql"

In [6]:
sql_path = os.path.join(dir)
sqld = SQLiteResult(sql_path)

In [7]:
sqld.available_outputs

('Site Outdoor Air Drybulb Temperature',
 'AFN Surface Venting Window or Door Opening Factor',
 'AFN Surface Opening Elapsed Time',
 'AFN Surface Opening Status at Previous Time Step',
 'AFN Surface Opening Probability Status',
 'AFN Surface Closing Elapsed Time',
 'Zone Mean Air Temperature',
 'AFN Surface Closing Probability Status',
 'Site Wind Direction',
 'Site Wind Speed',
 'AFN Linkage Node 2 to Node 1 Mass Flow Rate',
 'AFN Surface Venting Window or Door Opening Factor at Previous Time Step',
 'AFN Linkage Node 1 to Node 2 Mass Flow Rate',
 'AFN Surface Opening Status')

In [44]:
temp = sqld.data_collections_by_output_name("Zone Mean Air Temperature")
west_temp = temp[6]
east_temp = temp[7]
# for ix, t in enumerate(temp):
#     print(ix, "\n", t.header)

In [11]:
wo = sqld.data_collections_by_output_name("AFN Surface Venting Window or Door Opening Factor at Previous Time Step")
wind_1_ap = wo[4]

In [41]:
outdoor_temp = sqld.data_collections_by_output_name("Site Outdoor Air Drybulb Temperature")
ot = outdoor_temp[2]

In [47]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=wind_1_ap.datetimes,
    y=wind_1_ap.values, 
    mode='lines+markers',
    name="Window 1 Opening Factor"
),
secondary_y=False)

fig.add_trace(go.Scatter(
    x=west_temp.datetimes,
    y=west_temp.values, 
    mode='lines',
    name="West Zone Temp"
), secondary_y=True)

fig.add_trace(go.Scatter(
    x=east_temp.datetimes,
    y=east_temp.values, 
    mode='lines',
    name="East Zone Temp"
), secondary_y=True)


fig.add_trace(go.Scatter(
    x=ot.datetimes,
    y=ot.values, 
    mode='lines',
    name="Outdoor Temp"
), secondary_y=True)

fig.update_layout(title='AFNWindowOp Example',
                   xaxis_title='Dates',
                   yaxis_title='Temperature (ºC)')
fig.update_yaxes(title_text="Factor", secondary_y=True)
fig.update_yaxes(title_text="Temp [ºC]", secondary_y=True)